## Sesión 2.3.2 Clasificación de texto con scikit-learn y TFIDF

En esta sesión vamos a ver cómo podemos desarrollar un clasificador de texto sencillo con la librería scikit y la representación de texto de la bolsa de palabras usando TFIDF y BM25.

Vamos a usar el dataset de la sesión 2.1 que tiene etiquetado los tuits como positivos y negativos.

In [ ]:
# Instalamos las librerías necesarias y descargamos los recursos
!pip3 install -U scikit-learn
# Descargamos un fichero python con la implementación del BM25
!wget --no-check-certificate https://valencia.inf.um.es/dlpln/BM25.py

# Descargamos el fichero datasetEspañol.csv
!wget --no-check-certificate https://valencia.inf.um.es/dlpln/datasetEspañol.csv

In [ ]:
import pandas
df = pandas.read_csv("datasetEspañol.csv",encoding="UTF-8")
df.tail()

## Apartado 1 División de conjunto de entrenamiento y test (Resuelto)

Dividimos entre conjunto de entrenamiento, test y validación de manera aleatoria

In [ ]:
p_train = 0.80 # Porcentaje de train.
p_test = 0.20 # Porcentaje de train.

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size = p_test)

# Ponemos en lower_case los dos conjuntos de tweets
# Aquí se podría hacer un preprocesamiento como vimos en la sesión práctica 1
df_train.tweet = df_train.tweet.apply(lambda x: x.lower())
df_test.tweet = df_test.tweet.apply(lambda x: x.lower())

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para test: ", len(df_test))

## Apartado 2 Generación de los modelos de BoW (Resuelto)
Generamos los modelos de bolsa de palabras usando el CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
# Pueden haber distintas opciones para crear el CountVectorizer() como eliminar las stopwords, usar ngramas, etc.
X_train_counts = count_vect.fit_transform(df_train.tweet)
X_train_counts.shape

In [ ]:
# Consultamos cuál sería el índice de un término
count_vect.vocabulary_.get(u'coronavirus')


### Modelo de TF, TFIDF y BM25
Vamos a probar cómo funcionan los distintos modelos tanto de TF, TFIDF y BM25. Para ello generaremos 3 modelos distintos para entrenamiento y validación.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

# Generamos primeramente el TF usando el parámetro "use_idf=false"
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

In [ ]:
# Calculamos el TFIDF ahora y lo guardamos en X_train_tfidf
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
from BM25 import BM25Transformer
# Calculamos el BM25 ahora y lo guardamos en X_train_bm25
bm25_transformer = BM25fTransformer().fit(X_train_counts)
X_train_bm25 = bm25_transformer.transform(X_train_counts)
X_train_bm25.shape

### Entrenamos un mismo algoritmo de clasificación con los modelos
Entrenamos un algoritmo de clasificación el LinearSVM con TF, TFIDF y BM25

In [ ]:
from sklearn.svm import LinearSVC

# Entrenamos el modelo para TF y lo guardamos en clf_tf
clf_tf = LinearSVC(random_state=0, tol=1e-5, dual=True).fit(X_train_tf, df_train.label)

# Entrenamos el modelo para TFIDF y lo guardamos en clf_tfidf
clf_tfidf = LinearSVC(random_state=0, tol=1e-5, dual=True).fit(X_train_tfidf, df_train.label)

# Entrenamos el modelo para TF y lo guardamos en clf_bm25
clf_bm25 = LinearSVC(random_state=0, tol=1e-5, dual=True).fit(X_train_bm25, df_train.label)

Probamos ejemplos

In [ ]:
docs_new = ['no me fío del gobierno', 'aumentan los fallecidos','desciende la incidencia del virus']
X_new_counts = count_vect.transform(docs_new)
# Obtenemos los vectores para TF y los guardamos en X_new_tf
X_new_tf = tf_transformer.transform(X_new_counts)

# Obtenermos los vectores para TFIDF y los guardamos en X_new_tfidf
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

# Obtenemos los vectores para BM25 y los guardamos en X_new_bm25
X_new_bm25 = bm25_transformer.transform(X_new_counts)

# Predecimos entonces las categorías para TF y las guardamos en predicted_tf
predicted_tf = clf_tf.predict(X_new_tf)

# Predecimos entonces las categorías para TFIDF y las guardamos en predicted_tfidf
predicted_tfidf = clf_tfidf.predict(X_new_tfidf)

# Predecimos entonces las categorías para BM25 y las guardamos en predicted_bm25
predicted_bm25 = clf_bm25.predict(X_new_bm25)

# Imprimimos los textos y su predicción para TF
for doc, category_tf in zip(docs_new, predicted_tf):
  print('TF: %r => %s' % (doc, category_tf))

# Imprimimos los textos y su predicción para TFIDF
for doc, category_tfidf in zip(docs_new, predicted_tfidf):
  print('TFIDF: %r => %s' % (doc, category_tfidf))

# Imprimimos los textos y su predicción para BM25
for doc, category_bm25 in zip(docs_new, predicted_bm25):
  print('BM25: %r => %s' % (doc, category_bm25))

## Apartado 3 Creamos los modelos mediante pipeline
Para simplificar el código de creación de modelos se puede hacer creando un pipeline.

In [ ]:
from sklearn.pipeline import Pipeline
# Se pueden crear modelos mediante Pipeline
# Creamos el pipeline de TF con LinearSVC
clf_tf = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=False)),
    ('clf', LinearSVC(random_state=0, tol=1e-5)),])

# Creamos el pipeline de TFIDF con LinearSVC y lo guardamos en clf_tfidf
clf_tfidf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC(random_state=0, tol=1e-5)),])

# Creamos el pipeline de BM25 con LinearSVC y lo guardamos en clf_bm25
clf_bm25 = Pipeline([
    ('vect', CountVectorizer()),
    ('bm25', BM25Transformer()),
    ('clf', LinearSVC(random_state=0, tol=1e-5)),])

# Entrenamos el modelo de TF con el conjunto de entrenamiento con sus etiquetas
clf_tf.fit(df_train.tweet, df_train.label)

# Entrenamos el modelo de TFIDF con el conjunto de entrenamiento con sus etiquetas
clf_tfidf.fit(df_train.tweet, df_train.label)

# Entrenamos el modelo de BM25 con el conjunto de entrenamiento con sus etiquetas
clf_bm25.fit(df_train.tweet, df_train.label)

Evaluamos el accuracy del test en los 3 algoritmos

In [ ]:
import numpy as np

# Evaluamos el TF
predicted_tf = clf_tf.predict(df_test.tweet)
accuracy_tf = np.mean(predicted_tf == df_test.label)

print("Resultados TF ----- Accuracy:", accuracy_tf)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_tf))

# Evaluamos el TFIDF
predicted_tfidf = clf_tfidf.predict(df_test.tweet)
accuracy_tfidf = np.mean(predicted_tfidf == df_test.label)

print("Resultados TFIDF ----- Accuracy:", accuracy_tfidf)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_tfidf))

# Evaluamos el BM25
predicted_bm25 = clf_bm25.predict(df_test.tweet)
accuracy_bm25 = np.mean(predicted_bm25 == df_test.label)

print("Resultados BM25 ----- Accuracy:", accuracy_bm25)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_bm25))

## Apartado 4 Imprimimos las matrices de confusión
Para que se vean más bonitas podemos instalar las librerías matplotlib y seaborn

In [ ]:
!pip3 install -U matplotlib
!pip3 install -U seaborn

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

# Pintamos la matriz de confusión de TF
array_tf = metrics.confusion_matrix(df_test.label, predicted_tf)
df_cm_tf = pd.DataFrame(array_tf)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm_tf, annot=True, fmt="d")

# Pintamos la matriz de confusión de TFIDF
array_tfidf = metrics.confusion_matrix(df_test.label, predicted_tfidf)
df_cm_tfidf = pd.DataFrame(array_tfidf)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm_tfidf, annot=True, fmt="d")

# Pintamos la matriz de confusión de BM25
array_bm25 = metrics.confusion_matrix(df_test.label, predicted_bm25)
df_cm_bm25 = pd.DataFrame(array_bm25)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm_bm25, annot=True, fmt="d")

## Apartado 5 - Guardamos y cargamos el modelo (Resuelto)


In [ ]:
import joblib

# Podemos gardar el modelo a algún archivo
modTF_file = "TF_model.pkl"
joblib.dump(clf_tf, modTF_file)

# Lo cargamos desde el fichero
TF_model = joblib.load(modTF_file)

# Volvemos a calcular el accuracy de los datos de test
score = TF_model.score(df_test.tweet, df_test.label)
print("Test accuracy: {0:.8f} %".format(100 * score))

# Otra forma de calcular el accuracy es
predicted_tf = TF_model.predict(df_test.tweet)
np.mean(predicted_tf == df_test.label)

# Probamos de nuevo los ejemplos para la clasificación
docs_new = ['no me fío del gobierno', 'aumentan los fallecidos','desciende la incidencia del virus']

# Predecimos
predicted_tf = TF_model.predict(docs_new)

# Imprimimos los textos y su predicción para TF
for doc, category_tf in zip(docs_new, predicted_tf):
  print('TF: %r => %s' % (doc, category_tf))

# Apartado 6 Creamos modelos con otros algoritmos de clasificación de scikit-learn

En la documentación de scikit-learn se describen multitud de clasificadores que se pueden utilizar https://scikit-learn.org/stable/supervised_learning.html

In [ ]:
# Creamos y entrenamos modelos mediante Pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# Se pueden crear modelos mediante Pipeline
# Creamos el pipeline de TF con LinearSVC
clf_tf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(3,5), analyzer="char_wb", min_df=5)),
    ('tf', TfidfTransformer(use_idf=False)),
    ('clf', RandomForestClassifier(random_state=0)),])

# Creamos el pipeline de TFIDF con LinearSVC y lo guardamos en clf_tfidf
clf_tfidf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(3,5), analizer="char_wb", min_df=5)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(random_state=0)),])

# Creamos el pipeline de BM25 con LinearSVC y lo guardamos en clf_bm25
clf_bm25 = Pipeline([
    ('vect', CountVectorizer(ngram_range=(3,5), analizer="char_wb", min_df=5)),
    ('bm25', BM25Transformer()),
    ('clf', RandomForestClassifier(random_state=0)),])

# Entrenamos el modelo de TF con el conjunto de entrenamiento con sus etiquetas
clf_tf.fit(df_train.tweet, df_train.label)

# Entrenamos el modelo de TFIDF con el conjunto de entrenamiento con sus etiquetas
clf_tfidf.fit(df_train.tweet, df_train.label)

# Entrenamos el modelo de BM25 con el conjunto de entrenamiento con sus etiquetas
clf_bm25.fit(df_train.tweet, df_train.label)

In [ ]:
# Evaluamos los nuevos modelos
import numpy as np

# Evaluamos el TF
predicted_tf = clf_tf.predict(df_test.tweet)
accuracy_tf = np.mean(predicted_tf == df_test.label)

print("Resultados TF ----- Accuracy:", accuracy_tf)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_tf))

# Evaluamos el TFIDF
predicted_tfidf = clf_tfidf.predict(df_test.tweet)
accuracy_tfidf = np.mean(predicted_tfidf == df_test.label)

print("Resultados TFIDF ----- Accuracy:", accuracy_tfidf)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_tfidf))

# Evaluamos el BM25
predicted_bm25 = clf_bm25.predict(df_test.tweet)
accuracy_bm25 = np.mean(predicted_bm25 == df_test.label)

print("Resultados BM25 ----- Accuracy:", accuracy_bm25)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_bm25))

In [ ]:
# Creamos y entrenamos modelos mediante Pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# Se pueden crear modelos mediante Pipeline
# Creamos el pipeline de TF con LinearSVC
clf_tf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(3,5), analyzer="char_wb", min_df=5)),
    ('tf', TfidfTransformer(use_idf=False)),
    ('clf', LinearSVC(random_state=0, tol=1e-5, dual=True)),])

# Creamos el pipeline de TFIDF con LinearSVC y lo guardamos en clf_tfidf
clf_tfidf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(3,5), analizer="char_wb", min_df=5)),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC(random_state=0, tol=1e-5, dual=True)),])

# Creamos el pipeline de BM25 con LinearSVC y lo guardamos en clf_bm25
clf_bm25 = Pipeline([
    ('vect', CountVectorizer(ngram_range=(3,5), analizer="char_wb", min_df=5)),
    ('bm25', BM25Transformer()),
    ('clf', LinearSVC(random_state=0, tol=1e-5, dual=True)),])

# Entrenamos el modelo de TF con el conjunto de entrenamiento con sus etiquetas
clf_tf.fit(df_train.tweet, df_train.label)

# Entrenamos el modelo de TFIDF con el conjunto de entrenamiento con sus etiquetas
clf_tfidf.fit(df_train.tweet, df_train.label)

# Entrenamos el modelo de BM25 con el conjunto de entrenamiento con sus etiquetas
clf_bm25.fit(df_train.tweet, df_train.label)

In [ ]:
# Evaluamos los nuevos modelos
import numpy as np

# Evaluamos el TF
predicted_tf = clf_tf.predict(df_test.tweet)
accuracy_tf = np.mean(predicted_tf == df_test.label)

print("Resultados TF ----- Accuracy:", accuracy_tf)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_tf))

# Evaluamos el TFIDF
predicted_tfidf = clf_tfidf.predict(df_test.tweet)
accuracy_tfidf = np.mean(predicted_tfidf == df_test.label)

print("Resultados TFIDF ----- Accuracy:", accuracy_tfidf)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_tfidf))

# Evaluamos el BM25
predicted_bm25 = clf_bm25.predict(df_test.tweet)
accuracy_bm25 = np.mean(predicted_bm25 == df_test.label)

print("Resultados BM25 ----- Accuracy:", accuracy_bm25)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted_bm25))